## Práctica 4
## Regresión Polinomial
## Vianey Maravilla Pérez

**Especificaciones:**

**Utilizando el dataset cal_housing.csv realiza lo siguiente:**

- Las columnas Longitude, latitude, housingMedianAge, totalRooms, totalBedrooms, population, households, medianIncome son las características de las casas.

- La columna medianHouseValue es el valor a predecir (target)

**Con el dataset especificado realice lo siguiente**

- Carge el dataset
- Genere un conjunto de entrenamiento con 80% de los datos y 20% para pruebas.
- Mezcle los datos (set random_state=0)
- Con el conjunto de entrenamiento genere un conjunto de validación de 10 pliegues

**Usando el conjunto de entrenamiento (80% de los datos) realice lo siguiente:**

- Usando el conjunto de validación (80% de los datos en 10 pliegues) realice lo siguiente para cada pliegue
- **Usando los datos sin escalar:**
  - Utilice regresión lineal mediante gradiente descendiente estocástico     para generar una línea que se ajuste lo mejor posible a los datos       (modificando parámetros max_iter, learning_rate y eta0) y reporte el     error cuadrado medio (MSE) y el coeficiente de determinación (r^2)
  - Utilice regresión polinomial mediante gradiente descendiente estocástico con polinomios de grado 2 y 3 para generar una línea curva que se ajuste lo mejor posible a los datos (modificando parámetros max_iter, learning_rate y eta0) y reporte MSE y r^2
  - Almacene los valores MSE y r^2 de cada pliegue y al terminar los 10 pliegues reporte el resultado promedio estos.
  
**Usando el conjunto de validación (80% de los datos en 10 pliegues) realice lo siguiente para cada pliegue**

- Usando los datos con escalado estándar repita todos los experimentos anteriores
- Usando los datos con escalado robusto repita todos los experimentos anteriores 

- Usando el conjunto de entrenamiento (80% de los datos) entrene un modelo utilizando el tipo de regresión (lineal o polinomial) y el método de escalado(sin escalado, estándar o robusto) que mejor resultado obtuvo en promedio en el conjunto de validación
- Usando el modelo anterior realice la predcción de los datos del conjunto de prueba y reporte MSE y r^2

**EVIDENCIAS:**
**Un documento que contenga lo siguiente:**
- Una introducción donde se explique el problema a resilver y una descripción del método de regresión lineal
- Tabla que reporte los resultados en el conjunto de validación y algunas conclusiones que se pueden obtener de estas, indicando las características (tipo de regresión, escalamiento, etc.) que tendrá el modelo que usará en el conjunto de prueba

* Resultados del MSE y R^2 en el conjunto de prueba obtenidos por el modelo entrenado
* Conclusiones generales de la práctica realizada, mencionando las dificultades encontradas y alguna idea para mejorar los resultados obtenidos 

In [9]:
# Importación de librerias 
import pandas as pd
import numpy as np
import operator 
import os
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

In [10]:
# Creación del archivo a trabajar en un dataframe
dataframe = pd.read_csv('./cal_housing.csv',sep = ',', engine = 'python')

In [11]:
# Corpus 

X = dataframe.drop('medianHouseValue',axis = 1).values

# Etiquetas 

y = dataframe['medianHouseValue'].values

In [12]:
# Generación Data Test y Data Train 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [27]:
class validation_set:
    
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

############################################### VALIDATION SETS ##########################################################
validation_sets = []
kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(X_train):
    X_train_v, X_test_v = X_train[train_index], X_train[test_index]
    y_train_v, y_test_v = y_train[train_index], y_train[test_index]
    validation_sets.append(validation_set(X_train_v, y_train_v, X_test_v, y_test_v))
    
################################################ CREACIÓN DE LIST ########################################################

regression_type_list = list()
scale_type_list = list()
learning_rate_list = list()
eta0_list = list()
iterations_list = list()
mse_list = list()
r2_list = list()

In [14]:
# Definición de la regresión Lineal 

def linear_regression(X_train, X_test, y_train, y_test, eta0 = 0.001, learning_rate = 'constant', iterations = 10000, standar_scale = False, robust_scale = False):
    if standar_scale:
        X_train = preprocessing.StandardScaler().fit_transform(X_train)
        X_test = preprocessing.StandardScaler().fit_transform(X_test)
    if robust_scale:
        X_train = preprocessing.RobustScaler().fit_transform(X_train)
        X_test = preprocessing.RobustScaler().fit_transform(X_test)
    regr = SGDRegressor(learning_rate = learning_rate, eta0 = eta0, max_iter = iterations)
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    return [mse, r2]

In [15]:
# Definición de la regresión Polinomial 

def polynomial_regression(X_train, X_test, y_train, y_test, degree = 2, eta0 = 0.001, learning_rate = 'constant', iterations = 10000, standar_scale = False, robust_scale = False):
    polynomial_features= PolynomialFeatures(degree = degree)
    X_train = polynomial_features.fit_transform(X_train)
    X_test = polynomial_features.fit_transform(X_test)
    if standar_scale:
        X_train = preprocessing.StandardScaler().fit_transform(X_train)
        X_test = preprocessing.StandardScaler().fit_transform(X_test)
    if robust_scale:
        X_train = preprocessing.RobustScaler().fit_transform(X_train)
        X_test = preprocessing.RobustScaler().fit_transform(X_test)
    regr = SGDRegressor(learning_rate = learning_rate, eta0 = eta0, max_iter = iterations)
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    return [mse, r2]

In [16]:
# Definición para obtener los mejores parametros 

def get_best_params(validation_sets, eta0, learnin_rate, iterations, type_regression, scale_type):
    c = 1
    mses = list()
    r2s = list()
    n = len(validation_sets)
    for validation_set in validation_sets:
        if scale_type == 'none':
            if type_regression == 1:
                mse, r2 = linear_regression(validation_set.X_train, validation_set.X_test, validation_set.y_train, validation_set.y_test, eta0, learning_rate, iterations)
            else:
                mse, r2 = polynomial_regression(validation_set.X_train, validation_set.X_test, validation_set.y_train, validation_set.y_test, type_regression, eta0, learning_rate, iterations)
        elif scale_type == 'std':
            if type_regression == 1:
                mse, r2 = linear_regression(validation_set.X_train, validation_set.X_test, validation_set.y_train, validation_set.y_test, eta0, learning_rate, iterations, standar_scale = True)
            else:
                mse, r2 = polynomial_regression(validation_set.X_train, validation_set.X_test, validation_set.y_train, validation_set.y_test, type_regression, eta0, learning_rate, iterations, standar_scale = True)
        elif scale_type == 'robust':
            if type_regression == 1:
                mse, r2 = linear_regression(validation_set.X_train, validation_set.X_test, validation_set.y_train, validation_set.y_test, eta0, learning_rate, iterations, robust_scale = True)
            else:
                mse, r2 = polynomial_regression(validation_set.X_train, validation_set.X_test, validation_set.y_train, validation_set.y_test, type_regression, eta0, learning_rate, iterations, robust_scale = True)
        print('kfold:', c)
        print('\tmse:', mse)
        print('\tr2:', r2)
        c = c + 1
        mses.append(mse)
        r2s.append(r2)
    mses_mean = sum(mses) / n
    r2s_mean = sum(r2s) / n
    return [mses_mean, r2s_mean]

########################################### Tipos de escalamiento ########################################################

scale_types = ['none', 'std', 'robust']
type_resgressions = [1, 2, 3]
# learning_rates = ['constant', 'optimal', 'invscaling', 'adaptive']
learning_rates = ['constant']
eta0s = [0.0001, 0.00001, 0.000001]
iterationss = [200000, 300000]

############################################### Escalamiento #############################################################

for scale_type in scale_types:
    for type_resgression in type_resgressions:
        for learning_rate in learning_rates:
            for eta0 in eta0s:
                for iterations in iterationss:
                    print(scale_type)
                    print('\t', type_resgression)
                    print('\t\t', learning_rate)
                    print('\t\t\t', eta0)
                    print('\t\t\t\t', iterations)
                    mses_mean, r2s_mean = get_best_params(validation_sets, eta0, learning_rate, iterations, type_resgression, scale_type)
                    scale_type_list.append(scale_type)
                    regression_type_list.append(type_resgression)
                    learning_rate_list.append(learning_rate)
                    eta0_list.append(eta0)
                    iterations_list.append(iterations)
                    mse_list.append(mses_mean)
                    r2_list.append(r2s_mean)
                    print("")
                    print("")

none
	 1
		 constant
			 0.0001
				 200000
kfold: 1
	mse: 7.76450014331502e+28
	r2: -5.593107481371539e+18
kfold: 2
	mse: 4.84363090387345e+29
	r2: -3.6056628480638726e+19
kfold: 3
	mse: 1.411896576463116e+31
	r2: -1.0480522287955158e+21
kfold: 4
	mse: 4.583487105833695e+28
	r2: -3.3886494972226637e+18
kfold: 5
	mse: 8.088196548331054e+28
	r2: -5.921637426510903e+18
kfold: 6
	mse: 3.377526155526915e+29
	r2: -2.463190025736244e+19
kfold: 7
	mse: 1.5707556563990951e+31
	r2: -1.2313769843584986e+21
kfold: 8
	mse: 1.2091091193029305e+30
	r2: -9.017029685157066e+19
kfold: 9
	mse: 1.2354197792413281e+31
	r2: -9.765657863274036e+20
kfold: 10
	mse: 2.3699690431315474e+28
	r2: -1.7834823959575724e+18


none
	 1
		 constant
			 0.0001
				 300000
kfold: 1
	mse: 6.720532882679014e+28
	r2: -4.841092414336296e+18
kfold: 2
	mse: 1.190798099756037e+30
	r2: -8.86445840537064e+19
kfold: 3
	mse: 3.2647465614019784e+30
	r2: -2.423424609967687e+20
kfold: 4
	mse: 1.480499103148608e+30
	r2: -1.094558013512

kfold: 9
	mse: 2.346468866751553e+41
	r2: -1.8548199182622785e+31
kfold: 10
	mse: 4.472061901927338e+44
	r2: -3.365378842746866e+34


none
	 3
		 constant
			 0.0001
				 200000
kfold: 1
	mse: 2.7592438077007545e+63
	r2: -1.987603438589189e+53
kfold: 2
	mse: 3.257817698522505e+63
	r2: -2.4251625432346222e+53
kfold: 3
	mse: 2.1285002000138288e+63
	r2: -1.5799878091668922e+53
kfold: 4
	mse: 2.8557988405214303e+66
	r2: -2.111340357603514e+56
kfold: 5
	mse: 4.7070692624332073e+64
	r2: -3.446201801235292e+54
kfold: 6
	mse: 5.384322631568997e+64
	r2: -3.926723048383804e+54
kfold: 7
	mse: 4.4256873275560855e+65
	r2: -3.4694699286411394e+55
kfold: 8
	mse: 1.485430974846918e+65
	r2: -1.1077722416954725e+55
kfold: 9
	mse: 3.173292360177311e+64
	r2: -2.5084014365273902e+54
kfold: 10
	mse: 4.8143894949600454e+66
	r2: -3.6229920118276047e+56


none
	 3
		 constant
			 0.0001
				 300000
kfold: 1
	mse: 3.986075522009411e+63
	r2: -2.871343732623622e+53
kfold: 2
	mse: 1.5874874204309342e+63
	r2: -1.18

kfold: 3
	mse: 4679992175.385369
	r2: 0.6526037167364472
kfold: 4
	mse: 4301510379.298103
	r2: 0.6819820663277445
kfold: 5
	mse: 5194514820.118984
	r2: 0.6196923110414613
kfold: 6
	mse: 4328880117.587196
	r2: 0.6842998740128114
kfold: 7
	mse: 4163748555.7328424
	r2: 0.6735874151210018
kfold: 8
	mse: 4343973345.502279
	r2: 0.6760446515323031
kfold: 9
	mse: 4655736836.955087
	r2: 0.6319766461965994
kfold: 10
	mse: 3884509298.6523094
	r2: 0.7076774495785958


std
	 2
		 constant
			 0.0001
				 300000
kfold: 1
	mse: 4539266871.731726
	r2: 0.6730168454941241
kfold: 2
	mse: 4185100997.7606664
	r2: 0.6884555515790197
kfold: 3
	mse: 4614049876.048879
	r2: 0.6574986201552683
kfold: 4
	mse: 4304271742.610227
	r2: 0.6817779140702336
kfold: 5
	mse: 5087910424.737465
	r2: 0.6274971730245944
kfold: 6
	mse: 4321731767.12112
	r2: 0.6848211947889566
kfold: 7
	mse: 4177292516.2204304
	r2: 0.6725256509214885
kfold: 8
	mse: 4338677345.622166
	r2: 0.67643960503459
kfold: 9
	mse: 4692073790.066859
	r2: 0.6

kfold: 6
	mse: 4884144600.920947
	r2: 0.6438050895459266
kfold: 7
	mse: 4633622973.411333
	r2: 0.6367521160653518
kfold: 8
	mse: 4636816990.057999
	r2: 0.6542055983491557
kfold: 9
	mse: 5467927589.097086
	r2: 0.5677751728317748
kfold: 10
	mse: 4439722093.106531
	r2: 0.6658957964473338


robust
	 1
		 constant
			 1e-05
				 200000
kfold: 1
	mse: 5149121906.689364
	r2: 0.6290863323172984
kfold: 2
	mse: 4623645477.505928
	r2: 0.6558097210187995
kfold: 3
	mse: 5031581338.407506
	r2: 0.62650521830048
kfold: 4
	mse: 4624969777.171542
	r2: 0.6580681662628635
kfold: 5
	mse: 5529578955.487491
	r2: 0.5951611524275046
kfold: 6
	mse: 4894170036.545779
	r2: 0.6430739463394014
kfold: 7
	mse: 4622715711.240942
	r2: 0.6376071791392467
kfold: 8
	mse: 4634723206.477135
	r2: 0.6543617439641506
kfold: 9
	mse: 5445957731.681975
	r2: 0.5695118303989106
kfold: 10
	mse: 4443154958.73709
	r2: 0.6656374616206602


robust
	 1
		 constant
			 1e-05
				 300000
kfold: 1
	mse: 5146647404.935576
	r2: 0.629264581451

kfold: 6
	mse: 6.394590287724405e+22
	r2: -4663499345405.355
kfold: 7
	mse: 6.259805177732206e+24
	r2: -490730686916533.4
kfold: 8
	mse: 8.489747205958115e+21
	r2: -633129808992.0636
kfold: 9
	mse: 2.6036677912926934e+25
	r2: -2058128683596539.5
kfold: 10
	mse: 8.693652644513512e+22
	r2: -6542269610226.656


robust
	 3
		 constant
			 1e-05
				 300000
kfold: 1
	mse: 3.5198556188857466e+22
	r2: -2535505239481.4565
kfold: 2
	mse: 7.53930709270425e+23
	r2: -56123598234061.31
kfold: 3
	mse: 1.8259041617063447e+23
	r2: -13553704698661.352
kfold: 4
	mse: 7.490725982750251e+25
	r2: -5538020343282356.0
kfold: 5
	mse: 1.7277558151401393e+24
	r2: -126494743762337.84
kfold: 6
	mse: 5.419955967711601e+23
	r2: -39527100205427.62
kfold: 7
	mse: 9.439817042969109e+24
	r2: -740024293142755.0
kfold: 8
	mse: 3.069137031618417e+23
	r2: -22888339257478.81
kfold: 9
	mse: 2.7516411483511413e+25
	r2: -2175097603974380.8
kfold: 10
	mse: 1.7208826763370465e+22
	r2: -1295022805318.0305


robust
	 3
		 constant

In [17]:
# Tabulación

data = {'Tipos de escalamiento': scale_type_list,
        'Tipo de regresión': regression_type_list,
        'Learning': learning_rate_list,
        'eta0': eta0_list,
        'Iteraciones': iterations_list,
        'mse':mse_list,
        'r2':r2_list}
dataframe1 = pd.DataFrame(data)
dataframe1

,Tipos de escalamiento,Tipo de regresión,Learning,eta0,Iteraciones,mse,r2
0,none,1,constant,0.000100,200000,4.444001e+30,-3.423541e+20
1,none,1,constant,0.000100,300000,2.775623e+30,-2.152721e+20
2,none,1,constant,0.000010,200000,1.106627e+28,-8.278950e+17
3,none,1,constant,0.000010,300000,1.628804e+28,-1.239977e+18
4,none,1,constant,0.000001,200000,1.537665e+26,-1.135505e+16
5,none,1,constant,0.000001,300000,4.209922e+25,-3.093540e+15
6,none,2,constant,0.000100,200000,1.567744e+47,-1.161724e+37
7,none,2,constant,0.000100,300000,8.474547e+47,-6.264564e+37
8,none,2,constant,0.000010,200000,5.964571e+45,-4.488144e+35
9,none,2,constant,0.000010,300000,1.148664e+46,-8.492126e+35


In [18]:
# Tabla ordenada
dataframe1.to_csv('resultados.csv')
tabla_ordenar = dataframe1.sort_values('mse')
tabla_ordenar

,Tipos de escalamiento,Tipo de regresión,Learning,eta0,Iteraciones,mse,r2
35,std,3,constant,0.000001,300000,4.248470e+09,6.822577e-01
32,std,3,constant,0.000010,200000,4.255914e+09,6.816554e-01
33,std,3,constant,0.000010,300000,4.256485e+09,6.815860e-01
34,std,3,constant,0.000001,200000,4.256839e+09,6.816071e-01
28,std,2,constant,0.000001,200000,4.415692e+09,6.698453e-01
29,std,2,constant,0.000001,300000,4.415992e+09,6.698188e-01
25,std,2,constant,0.000100,300000,4.417605e+09,6.696469e-01
26,std,2,constant,0.000010,200000,4.429023e+09,6.688475e-01
24,std,2,constant,0.000100,200000,4.432607e+09,6.685705e-01
27,std,2,constant,0.000010,300000,4.435617e+09,6.683401e-01


In [19]:
mse, r2 = polynomial_regression(X_train, X_test, y_train, y_test, degree = 3, eta0 = 0.00001, learning_rate = 'constant', iterations = 200000, standar_scale = True)

In [20]:
print('mse con X_train y X_test:', mse)
print('r2 con X_train y X_test:', r2)

mse con X_train y X_test: 4294888897.376936
r2 con X_train y X_test: 0.6706259633862597
